In [19]:
import pandas as pd
import geopandas as gpd
import time
import datetime as dt


Scooters_2023_05_17 = pd.read_csv('Data/Scooters 2023-05-17.csv')


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [20]:
Scooters_2023_05_17.head()

,Unnamed: 0,title,id,networkName,networkId,color,textColor,latitude,longitude,type,subtitle,stationProperties,time
0,0,Link Scooter,8c088413-0b87-49a6-8ece-b91b408545c1,LINK,1319,E5FF01,07000E,34.06379,-118.44610,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399
1,1,Link Scooter,76f5520c-94b7-46df-af20-c41adf63dcf1,LINK,1319,E5FF01,07000E,34.06743,-118.44965,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399
2,2,Link Scooter,5e260c79-05bb-4aeb-80e1-fe1c5d5e8639,LINK,1319,E5FF01,07000E,34.06710,-118.45234,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399
3,3,Link Scooter,0262bef4-168c-4db8-ab0a-cf5b20214107,LINK,1319,E5FF01,07000E,34.06964,-118.45061,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399
4,4,Gateway Plaza,bcycle_lametro_4613,Metro Bike Share,554,383838,FFFFFF,34.06975,-118.44535,station,2 bikes - 1 e-bike - 9 docks,"{'counts': {'bike': 2, 'electric-bike': 1, 'do...",2023-05-17 09:00:19.484399


Okay this is kind of an insane way to fix things, so I need to right this down for future readers. 

The problem at hand is some of the first data sets did not include a time variable, so its impossible to know when the data was taken. But our first unamed column always resets to zero after each call to the API. 

'Scooters 2023-05-17.csv' is the most complete dataset that I chose. The goal is to merge the time column onto the data that is missing that info. However since not all the calls ot the API started at 9am, but they all ended at 9:59pm.

To account for this, I made a column called 'cycle' which tells you which # call to the API the Data is from. But to account for some of the missing morning data, we take the largest number of calls possible (75) and subtract it by the number of call we are on.

We create a reverse_cycle column for each of the datasets missing its time column, and merge it with our complete dataset's column and we're done!

In [40]:
#Scooters_2023_05_17['cycle'] = Scooters_2023_05_17.groupby(['Unnamed: 0']).cumcount() + 1
Scooters_2023_05_17['reverse_cycle'] = Scooters_2023_05_17.groupby(['Unnamed: 0']).cumcount() - 75 
#Scooters_2023_05_11['reverse_cycle'] = Scooters_2023_05_11.groupby(['Unnamed: 0']).cumcount() - 75 
#Scooters_2023_05_17.assign(reverse_cycle=lambda x: n x.temp_c * 9 / 5 + 32)
print(Scooters_2023_05_17.max())
Scooters_2023_05_17.head()

Unnamed: 0                                        684
title                                 Wyton Dr Corral
id               ffff7fc0-19ca-4eb1-a87f-fbf43c495319
networkName                          Metro Bike Share
networkId                                        1340
color                                          EC39BA
textColor                                      FFFFFF
latitude                                    34.077785
longitude                                 -118.433907
type                                          station
time                       2023-05-17 21:59:24.977470
reverse_cycle                                      -1
dtype: object


C:\Users\Matucoco\AppData\Local\Temp\ipykernel_21264\4006304367.py:5: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(Scooters_2023_05_17.max())


,Unnamed: 0,title,id,networkName,networkId,color,textColor,latitude,longitude,type,subtitle,stationProperties,time,reverse_cycle
0,0,Link Scooter,8c088413-0b87-49a6-8ece-b91b408545c1,LINK,1319,E5FF01,07000E,34.06379,-118.44610,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399,-75
1,1,Link Scooter,76f5520c-94b7-46df-af20-c41adf63dcf1,LINK,1319,E5FF01,07000E,34.06743,-118.44965,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399,-75
2,2,Link Scooter,5e260c79-05bb-4aeb-80e1-fe1c5d5e8639,LINK,1319,E5FF01,07000E,34.06710,-118.45234,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399,-75
3,3,Link Scooter,0262bef4-168c-4db8-ab0a-cf5b20214107,LINK,1319,E5FF01,07000E,34.06964,-118.45061,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399,-75
4,4,Gateway Plaza,bcycle_lametro_4613,Metro Bike Share,554,383838,FFFFFF,34.06975,-118.44535,station,2 bikes - 1 e-bike - 9 docks,"{'counts': {'bike': 2, 'electric-bike': 1, 'do...",2023-05-17 09:00:19.484399,-75


In [41]:
Time_Cycle = pd.DataFrame()
Time_Cycle['reverse_cycle'] = Scooters_2023_05_17['reverse_cycle']
Time_Cycle['time'] = Scooters_2023_05_17['time']
Time_Cycle = Time_Cycle.drop_duplicates(subset=['reverse_cycle'])
Time_Cycle.head()

,reverse_cycle,time
0,-75,2023-05-17 09:00:19.484399
493,-74,2023-05-17 09:10:51.036629
1014,-73,2023-05-17 09:21:22.380851
1549,-72,2023-05-17 09:31:53.749638
2091,-71,2023-05-17 09:42:25.160216


In [42]:
Time_Cycle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 46674
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   reverse_cycle  75 non-null     int64 
 1   time           75 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


In [43]:
Scooters_2023_05_11 = pd.read_csv('Data/Scooters 2023-05-11.csv')
Scooters_2023_05_11['reverse_cycle'] = Scooters_2023_05_11.groupby(['Unnamed: 0']).cumcount() - 75 

Scooters_2023_05_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50393 entries, 0 to 50392
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         50393 non-null  int64  
 1   title              50393 non-null  object 
 2   id                 50393 non-null  object 
 3   networkName        50393 non-null  object 
 4   networkId          50393 non-null  int64  
 5   color              50393 non-null  object 
 6   textColor          50393 non-null  object 
 7   latitude           50393 non-null  float64
 8   longitude          50393 non-null  float64
 9   type               50393 non-null  object 
 10  subtitle           221 non-null    object 
 11  stationProperties  221 non-null    object 
 12  reverse_cycle      50393 non-null  int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 5.0+ MB


In [44]:
#testing cell, please ignore

#Scooters_2023_05_11 = pd.read_csv('Data/Scooters 2023-05-11.csv')
#Scooters_2023_05_11['reverse_cycle'] = Scooters_2023_05_11.groupby(['Unnamed: 0']).cumcount() - 75 



Scooters_2023_05_11_merge = Scooters_2023_05_11.merge(Time_Cycle, how = 'left', on = 'reverse_cycle')
Scooters_2023_05_11_merge.info()
Scooters_2023_05_11_merge.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50393 entries, 0 to 50392
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         50393 non-null  int64  
 1   title              50393 non-null  object 
 2   id                 50393 non-null  object 
 3   networkName        50393 non-null  object 
 4   networkId          50393 non-null  int64  
 5   color              50393 non-null  object 
 6   textColor          50393 non-null  object 
 7   latitude           50393 non-null  float64
 8   longitude          50393 non-null  float64
 9   type               50393 non-null  object 
 10  subtitle           221 non-null    object 
 11  stationProperties  221 non-null    object 
 12  reverse_cycle      50393 non-null  int64  
 13  time               50393 non-null  object 
dtypes: float64(2), int64(3), object(9)
memory usage: 5.8+ MB


,Unnamed: 0,title,id,networkName,networkId,color,textColor,latitude,longitude,type,subtitle,stationProperties,reverse_cycle,time
0,0,Link Scooter,decba7fc-debf-472a-8d51-2b7244d063e6,LINK,1319,E5FF01,07000E,34.06588,-118.44820,electric-scooter,NaN,NaN,-75,2023-05-17 09:00:19.484399
1,1,Link Scooter,808f4162-124f-48cc-a026-f17f61c2dcad,LINK,1319,E5FF01,07000E,34.06377,-118.44517,electric-scooter,NaN,NaN,-75,2023-05-17 09:00:19.484399
2,2,Link Scooter,688ad54a-72e5-43d2-ad6f-690fac0c02ae,LINK,1319,E5FF01,07000E,34.06960,-118.43841,electric-scooter,NaN,NaN,-75,2023-05-17 09:00:19.484399
3,3,Link Scooter,bb467505-a28f-4f80-ae8c-0b1fc3b6e584,LINK,1319,E5FF01,07000E,34.06377,-118.44513,electric-scooter,NaN,NaN,-75,2023-05-17 09:00:19.484399
4,4,Link Scooter,b7cc8424-a9d2-4fe7-ba70-859639d39584,LINK,1319,E5FF01,07000E,34.06563,-118.44002,electric-scooter,NaN,NaN,-75,2023-05-17 09:00:19.484399


In [45]:
#Ignore my ugly code, I know I should do a for loop but I'm lazy
Scooters_2023_05_10 = pd.read_csv('Data/Scooters 2023-05-10.csv')
Scooters_2023_05_11 = pd.read_csv('Data/Scooters 2023-05-11.csv')
Scooters_2023_05_12 = pd.read_csv('Data/Scooters 2023-05-12.csv')
Scooters_2023_05_15 = pd.read_csv('Data/Scooters 2023-05-15.csv')
Scooters_2023_05_16 = pd.read_csv('Data/Scooters 2023-05-16.csv')

Scooters_2023_05_10['reverse_cycle'] = Scooters_2023_05_10.groupby(['Unnamed: 0']).cumcount() - 75 
Scooters_2023_05_11['reverse_cycle'] = Scooters_2023_05_11.groupby(['Unnamed: 0']).cumcount() - 75 
Scooters_2023_05_12['reverse_cycle'] = Scooters_2023_05_12.groupby(['Unnamed: 0']).cumcount() - 75 
Scooters_2023_05_15['reverse_cycle'] = Scooters_2023_05_15.groupby(['Unnamed: 0']).cumcount() - 75 
Scooters_2023_05_16['reverse_cycle'] = Scooters_2023_05_16.groupby(['Unnamed: 0']).cumcount() - 75 

Scooters_2023_05_10 = Scooters_2023_05_10.merge(Time_Cycle,how = 'right', on = 'reverse_cycle')
Scooters_2023_05_11 = Scooters_2023_05_11.merge(Time_Cycle,how = 'right', on = 'reverse_cycle')
Scooters_2023_05_12 = Scooters_2023_05_12.merge(Time_Cycle,how = 'right', on = 'reverse_cycle')
Scooters_2023_05_15 = Scooters_2023_05_15.merge(Time_Cycle,how = 'right', on = 'reverse_cycle')
Scooters_2023_05_16 = Scooters_2023_05_16.merge(Time_Cycle,how = 'right', on = 'reverse_cycle')

#Scooters_2023_05_10.to_csv('Scooters 2023-05-10.csv')
#Scooters_2023_05_11.to_csv('Scooters 2023-05-11.csv')
#Scooters_2023_05_12.to_csv('Scooters 2023-05-12.csv')
#Scooters_2023_05_15.to_csv('Scooters 2023-05-15.csv')
#Scooters_2023_05_16.to_csv('Scooters 2023-05-16.csv')